# Data Processing

In [3]:
import pandas as pd
from matplotlib import pyplot as plt

In [4]:
path = 'C:\\Users\\Elly\\Dropbox\\Biotronik\\data\\200603 C1503 BIOSOLVE-IV - csv datasets'
ae_data = pd.read_csv('{}/{}.csv'.format(path, "AE"))
enrol = pd.read_csv('{}/{}.csv'.format(path, "ENROL"))

# get all available data for patients (baseline and information until hospital discharge)
file_abr_baseline = ["IE","EQ5D","MHRF","PCISUB","ISBP","ECG","CEBP","SCBP","LC","CMPROC"]
file_abr_procedure = ["LC","AE","AEACT","MHRFSUB","DEVDEF","DEVICE"]

# delete useless columns from enrol information
enrol.drop(
    [
        "visit","enstudyid","encipv",
        "ecrfvers","parent_record_id",
        "record_id","record_status",
        "record_status_text", "endtc",
        "interval_name","entim"
    ],
    axis=1,
    inplace=True
)



procedure_data = enrol

## Process baseline data

In [5]:
# process baseline data

baseline_data = enrol


for file in file_abr_baseline:
    df = pd.read_csv('{}/{}.csv'.format(path, file))
    # cast EQ5D table
    if file is 'EQ5D':
        df = df[df['visit']=="Screening 2.0"]
    # cast ECG
    elif file is 'ECG':
        df = df[
            (df["visit"] == "Screening") |
            (df["visit"] == "Screening 2.0")
        ]
    # cast LC
    elif file is 'LC':
        df = df[
            [
                "siteid","studyid","subjid",
                "lccassmap","lcid","lclesloc",
                "lcvessel","lctimipr","lcaccaha",
                "lccalccs","lcvangul","lctortsy",
                "lcbifurc","lcecntry","lcthrmbs",
                "lclenv","lcrvdv","lcdsv"
            ]
        ]
    # cast CMPROC
    elif file is 'CMPROC':
        df = df[
            [
                "siteid","studyid","subjid",
                "cmasapp","cmasappd","cmasappu",
                "cmasappdut","cmheppp","cmhepppd",
                "cmhepppu","cmheppput","cmplt",
                "cmpltt","cmpltpp","cmpltppd",
                "cmpltppu","cmpltpput","cmpltld",
                "cmpltldd","cmpltldu","cmpltldut"
            ]
        ]
    
    df.drop(
        [
            "record_id","record_status","record_status_text",
            "parent_record_id","interval_name","visit"
        ],
        axis=1,
        inplace=True,
        errors='ignore'
    )  
    baseline_data = pd.merge(
        baseline_data,
        df,
        on=("studyid","subjid","siteid"),
        how='outer'
    )


Attention: The above dataframe can contain for ONE patient multiple rows with more less the same information. That occurs because on patient can have for example two target lesions that are characterized. When working with data: always group by "subjid".